# Stock Management System - Optimización de Producción

Este notebook implementa un sistema para gestionar el inventario y optimizar la producción utilizando programación lineal.

In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from typing import Dict, Any

In [ ]:
class StockManagementSystem:
    def __init__(self, data_path: str):
        """
        Inicializar el sistema de gestión de inventario con los datos de entrada
        """
        try:
            # Cargar datos desde el archivo CSV
            self.df = pd.read_csv(data_path, sep=';', encoding='latin1', skiprows=3, header=0)
            self.df.columns = self.df.iloc[0]  # Primera fila como encabezado
            self.df = self.df[1:]  # Eliminar fila usada como encabezado
            self.df.columns = self.df.columns.str.strip()  # Limpiar espacios en los nombres
            print("Datos cargados correctamente. Columnas disponibles:", self.df.columns)
        except Exception as e:
            raise Exception(f"Error al cargar el archivo CSV: {e}")


In [ ]:
def _clean_data(self):
    """Limpieza y preparación de los datos"""
    try:
        # Eliminar filas completamente vacías y resetear índices
        self.df = self.df.dropna(how='all').reset_index(drop=True)
        self.df = self.df[self.df['COD_ART'] != 'Total general']

        # Convertir columnas numéricas relevantes
        num_columns = ['Cj/H', 'Disponible', 'Calidad', 'Stock Externo', 'M_Vta -15', 'M_Vta -15 AA']
        for col in num_columns:
            self.df[col] = self.df[col].replace({',': '.'}, regex=True)
            self.df[col] = pd.to_numeric(self.df[col], errors='coerce')

        # Manejar valores nulos
        self.df['Cj/H'] = self.df['Cj/H'].fillna(1)
        self.df['Disponible'] = self.df['Disponible'].fillna(0)
        self.df['Calidad'] = self.df['Calidad'].fillna(0)
        self.df['Stock Externo'] = self.df['Stock Externo'].fillna(0)
        self.df['M_Vta -15'] = self.df['M_Vta -15'].fillna(0)
        self.df['M_Vta -15 AA'] = self.df['M_Vta -15 AA'].fillna(1)

        # Calcular el stock total
        self.df['STOCK_TOTAL'] = (
            self.df['Disponible'] +
            self.df['Calidad'] +
            self.df['Stock Externo']
        ).fillna(0)

        print("Datos después de la limpieza:")
        print(self.df.head())
    except Exception as e:
        raise Exception(f"Error al limpiar los datos: {e}")


In [ ]:
system = StockManagementSystem("stock_data.csv")
system._clean_data()


In [ ]:
def _calculate_demand_estimation(self):
    """Cálculo de la estimación de demanda"""
    try:
        # Demanda media basada en las ventas
        self.df['DEMANDA_MEDIA'] = self.df['M_Vta -15'].fillna(0)

        # Calcular variación interanual
        self.df['VAR_ANUAL'] = abs(1 - (
            self.df['M_Vta -15'].fillna(0) /
            self.df['M_Vta -15 AA'].fillna(1)
        )) * 100

        # Ajustar tendencia para variaciones significativas
        mask = self.df['VAR_ANUAL'] > 20
        self.df.loc[mask, 'DEMANDA_MEDIA'] *= (1 + self.df.loc[mask, 'VAR_ANUAL'] / 100)

        print("Demanda calculada:")
        print(self.df[['COD_ART', 'DEMANDA_MEDIA', 'VAR_ANUAL']].head())
    except Exception as e:
        raise Exception(f"Error al calcular la estimación de demanda: {e}")


In [ ]:
def optimize_production(self, available_hours: float, maintenance_hours: float, pending_orders: Dict[str, float] = None):
    try:
        # Ajustar horas disponibles
        net_hours = available_hours - maintenance_hours
        production_rates = self.df['Cj/H'].fillna(1).values
        current_stock = self.df['STOCK_TOTAL'].values
        demand = self.df['DEMANDA_MEDIA'].values

        # Función objetivo
        c = -1 * np.maximum(demand - current_stock, 0)

        # Restricciones
        A_ub = [1 / np.where(production_rates == 0, 1, production_rates)]
        b_ub = [net_hours]

        min_production = self.MIN_PRODUCTION_HOURS * production_rates
        A_ub.extend(-1 * np.eye(len(production_rates)))
        b_ub.extend(-1 * min_production)

        # Resolver problema de optimización
        result = linprog(
            c,
            A_ub=np.array(A_ub),
            b_ub=np.array(b_ub),
            bounds=(0, None),
            method='highs'
        )

        # Crear plan de producción
        production_plan = self.df.copy()
        production_plan['CAJAS_PRODUCIR'] = result.x
        production_plan['HORAS_PRODUCCION'] = np.divide(
            result.x, 
            production_plan['Cj/H'].fillna(1),
            out=np.zeros_like(result.x),
            where=production_plan['Cj/H'].fillna(1) > 0
        )

        print("Plan de producción generado:")
        print(production_plan[['COD_ART', 'CAJAS_PRODUCIR', 'HORAS_PRODUCCION']].head())
        return production_plan
    except Exception as e:
        raise Exception(f"Error al optimizar la producción: {e}")


In [ ]:
production_plan = system.optimize_production(100, 5)


In [ ]:
def display_summary_table(self, production_plan: pd.DataFrame):
    try:
        summary_table = production_plan[['COD_ART', 'NOM_ART', 'CAJAS_PRODUCIR', 'HORAS_PRODUCCION', 'STOCK_TOTAL']].head(10)
        print("\nResumen del plan de producción:\n")
        print(summary_table)
    except Exception as e:
        raise Exception(f"Error al mostrar la tabla resumen: {e}")


In [ ]:
system.display_summary_table(production_plan)
